In [4]:
import pandas as pd 
import numpy as np
import seaborn as sns
import json
import sys
import argparse
import os
import re
import spacy
import html
import string
import csv
from matplotlib import pyplot as plt
from emfdscore.scoring import score_docs 

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import gensim
from gensim import corpora
from gensim.models import LdaModel
from collections import defaultdict

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /u/tiechu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /u/tiechu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /u/tiechu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [5]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])  
nlp.add_pipe('sentencizer')  

In [6]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
nlp.add_pipe('sentencizer')

def preproc(text):
    
    modComm = text
    #replace newlines with spaces
    modComm = re.sub(r"\s+", " ", modComm)

    modComm = html.unescape(modComm)

    modComm = re.sub(r"(http|www)\S+", "", modComm)
        
    modComm = ' '.join(modComm.split())

    # get Spacy document for modComm
    # use Spacy document for modComm to create a string.
    doc = nlp(modComm)
    modComm = ""
    for sentence in doc.sents:
        for token in sentence:
            # Replace token with token.lemma
            if token.text.startswith("-") and (not token.lemma_.startswith("-")):
                content = token.lemma_
            else:
                content = token.text
            
            # Uppercase distinction
            if token.text.isupper():
                modComm += content.upper()
            else:
                modComm += content.lower()    

            # Write "/POS" after each token.
            # Add the part-of-speech (pos) tag after the token text
            # modComm += "/" + token.tag_
            # Add a space after the token
            # Split tokens with spaces.
            modComm += " "
        # Insert "\n" between sentences.
        # Add a newline character after the sentence
        modComm += "\\n"
    return modComm
    # print(token.text, token.pos_, token.dep_)   
    # print(modComm)

directory = '/u/tiechu/anaconda3/envs/emfd/NELA2021_0'
files = os.listdir(directory)
index = 0
limit = len(files)
# limit = 5
output_data = pd.DataFrame(columns=['source', 'body'])
output_dict = {'source': [], 'body': []}

while index < limit:
    filename = files[index]
    if filename.endswith('.txt'):
        with open(os.path.join(directory, filename)) as f:
            name = filename.split('--')[0]
            body = preproc(f.read())
        output_dict['source'].append(name)
        output_dict['body'].append(body)
        index += 1


output_data = pd.DataFrame.from_dict(output_dict)
output_data.to_csv("output.csv")

In [7]:
with open('/u/tiechu/anaconda3/envs/emfd/output.csv', 'r') as file:
    # Load the JSON data
    csv_data = pd.read_csv(file)

csv_data

,Unnamed: 0,source,body
0,0,deneenborelli,NaN
1,1,themichellemalkinblog,"“ one is the loneliest number , ” three dog ni..."
2,2,upi,feb. 15 ( UPI ) -- twenty - three time grand s...
3,3,theborowitzreport,"FLORENCE , COLORADO ( the borowitz report ) — ..."
4,4,yahoonews,• none sen. ben sasse on sunday said that the ...
...,...,...,...
245,245,theborowitzreport,ATLANTA ( the borowitz report ) — president bi...
246,246,theborowitzreport,"WELLINGTON , OHIO ( the borowitz report ) — ho..."
247,247,deneenborelli,president biden flip - flopped on the decision...
248,248,rightwingwatch,right - wing pastor and self - proclaimed “ pr...


In [8]:
mfd_dict = {}
mfd_lst = [0, 44, 45, 46, 47,48,49,50,51,52,53]

with open('emfd_amp.csv', 'r') as file:
        reader = csv.reader(file, delimiter=',', quotechar='|')
        headers = next(reader, None)
        # print(headers)
        for row in reader:
            row_dict = {headers[i]: row[i] for i in mfd_lst}
            mfd_dict[row_dict[headers[0]]] = row_dict


file_path = '/u/tiechu/anaconda3/envs/emfd/output.csv'

# Open the JSON file
with open(file_path, 'r') as file:
    # Load the JSON data
    csv_data = pd.read_csv(file)

mfd_score = pd.DataFrame()
csv_data.fillna(' ', inplace=True)

for i in range(len(csv_data)):
    care_virtue = 0.0
    fairness_virtue = 0.0
    loyalty_virtue = 0.0
    authority_virtue = 0.0
    sanctity_virtue = 0.0
    
    care_vice = 0.0
    fairness_vice = 0.0
    loyalty_vice = 0.0
    authority_vice = 0.0
    sanctity_vice = 0.0
    # print(json_data[i])
    # print(json_data[i]['body'])
    word_length = 0
    
    for token in csv_data['body'][i].split():
        if token in mfd_dict:
            token_c = mfd_dict[token]['care.virtue']
            token_f = mfd_dict[token]['fairness.virtue']
            token_l = mfd_dict[token]['loyalty.virtue']
            token_a = mfd_dict[token]['authority.virtue']
            token_s = mfd_dict[token]['sanctity.virtue']

            decimal_pattern = re.compile(r'\d+\.\d+')
            c = decimal_pattern.search(token_c)
            f = decimal_pattern.search(token_f)
            l = decimal_pattern.search(token_l)
            a = decimal_pattern.search(token_a)
            s = decimal_pattern.search(token_s)

            if c:
                care_virtue += float(c.group())
            if f:
                fairness_virtue += float(f.group())  
            if l:
                loyalty_virtue += float(l.group())  
            if a:
                authority_virtue += float(a.group())  
            if s:
                sanctity_virtue += float(s.group())
            
            token_c = mfd_dict[token]['care.vice']
            token_f = mfd_dict[token]['fairness.vice']
            token_l = mfd_dict[token]['loyalty.vice']
            token_a = mfd_dict[token]['authority.vice']
            token_s = mfd_dict[token]['sanctity.vice']

            decimal_pattern = re.compile(r'\d+\.\d+')
            c = decimal_pattern.search(token_c)
            f = decimal_pattern.search(token_f)
            l = decimal_pattern.search(token_l)
            a = decimal_pattern.search(token_a)
            s = decimal_pattern.search(token_s)

            if c:
                care_vice += float(c.group())
            if f:
                fairness_vice += float(f.group())  
            if l:
                loyalty_vice += float(l.group())  
            if a:
                authority_vice += float(a.group())  
            if s:
                sanctity_vice += float(s.group())
            word_length+=1

    if word_length == 0:
        word_length = 1
    
    care_virtue = care_virtue/word_length*100
    fairness_virtue = fairness_virtue/word_length*100
    loyalty_virtue = loyalty_virtue/word_length*100
    authority_virtue = authority_virtue/word_length*100
    sanctity_virtue = sanctity_virtue/word_length*100

    care_vice = care_vice/word_length*100
    fairness_vice = fairness_vice/word_length*100
    loyalty_vice = loyalty_vice/word_length*100
    authority_vice = authority_vice/word_length*100
    sanctity_vice = sanctity_vice/word_length*100

    mfd_score[i] = [care_virtue,fairness_virtue,loyalty_virtue, 
                    authority_virtue,sanctity_virtue, care_vice,
                    fairness_vice, loyalty_vice, authority_vice,
                    sanctity_vice]
mfd_score = mfd_score.transpose()

/h/224/tiechu/anaconda3/envs/emfd/lib/python3.7/site-packages/ipykernel_launcher.py:108: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`


In [9]:
mfd_score = mfd_score.reset_index(drop=True)
csv_data = csv_data.reset_index(drop=True)

all_dataset = pd.concat([csv_data,mfd_score], axis=1, join='inner')
# all_dataset

In [10]:
copy_mfd = all_dataset.copy()
copy_mfd.columns = copy_mfd.columns.astype(str)

copy_mfd.rename(columns={'0': 'care_virtue', 
                          '1': 'fairness_virtue', 
                          '2': 'loyalty_virtue', 
                          '3': 'authority_virtue', 
                          '4': 'sanctity_virtue', 
                          '5': 'care_vice', 
                          '6': 'fairness_vice', 
                          '7': 'loyalty_vice', 
                          '8': 'authority_vice', 
                          '9': 'sanctity_vice'}, inplace=True)
copy_mfd

,Unnamed: 0,source,body,care_virtue,fairness_virtue,loyalty_virtue,authority_virtue,sanctity_virtue,care_vice,fairness_vice,loyalty_vice,authority_vice,sanctity_vice
0,0,deneenborelli,,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,1,themichellemalkinblog,"“ one is the loneliest number , ” three dog ni...",40.653438,40.653438,0.0,3.899043,4.738112,4.261840,4.153984,4.045030,8.452257,8.566532
2,2,upi,feb. 15 ( UPI ) -- twenty - three time grand s...,42.903333,42.903333,0.0,4.152343,8.074776,5.556080,6.149776,4.992699,3.428063,2.152295
3,3,theborowitzreport,"FLORENCE , COLORADO ( the borowitz report ) — ...",41.921875,41.921875,0.0,3.630889,5.133659,5.282277,3.553495,4.097359,9.357847,10.666996
4,4,yahoonews,• none sen. ben sasse on sunday said that the ...,40.188500,40.188500,0.0,5.814619,6.304806,5.182654,5.754733,4.614905,8.495536,7.279321
...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,245,theborowitzreport,ATLANTA ( the borowitz report ) — president bi...,41.743000,41.743000,0.0,6.694711,3.879984,5.066907,4.406429,4.028397,12.651327,6.916868
246,246,theborowitzreport,"WELLINGTON , OHIO ( the borowitz report ) — ho...",23.266667,23.266667,0.0,5.819775,10.039607,11.211203,8.059291,6.255706,8.071219,7.168476
247,247,deneenborelli,president biden flip - flopped on the decision...,29.381667,29.381667,0.0,7.535979,5.877512,4.637794,6.587134,4.300400,7.955576,6.829192
248,248,rightwingwatch,right - wing pastor and self - proclaimed “ pr...,37.983929,37.983929,0.0,3.913609,5.162972,5.320155,4.980907,4.215590,11.476372,7.765704


In [11]:
copy_mfd.to_csv('sample_output.csv')

In [12]:
orientation_dataframe = pd.read_excel('2021_orientation.xlsx')

orientation_dataframe = orientation_dataframe[['source', 'Media Bias/Fact Check Label (2021)']]

In [13]:
orientation_merge = pd.merge(copy_mfd, orientation_dataframe, on='source', how='left')

In [14]:
distinct_labels = ['left-bias', 'left-center', 'pro-science', 'center', 'right-center', 'right', 'questionable-source', 'conspiracy-pseudoscience']
# start left_bias as 1 and add more outlet labels

label_to_number = {label: number for number, label in enumerate(distinct_labels, start=1)}

orientation_merge['label_number'] = orientation_merge['Media Bias/Fact Check Label (2021)'].map(label_to_number)
display(orientation_merge)
orientation_merge.to_csv('orientation_merge.csv')

,Unnamed: 0,source,body,care_virtue,fairness_virtue,loyalty_virtue,authority_virtue,sanctity_virtue,care_vice,fairness_vice,loyalty_vice,authority_vice,sanctity_vice,Media Bias/Fact Check Label (2021),label_number
0,0,deneenborelli,,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN,NaN
1,1,themichellemalkinblog,"“ one is the loneliest number , ” three dog ni...",40.653438,40.653438,0.0,3.899043,4.738112,4.261840,4.153984,4.045030,8.452257,8.566532,NaN,NaN
2,2,upi,feb. 15 ( UPI ) -- twenty - three time grand s...,42.903333,42.903333,0.0,4.152343,8.074776,5.556080,6.149776,4.992699,3.428063,2.152295,center,4.0
3,3,theborowitzreport,"FLORENCE , COLORADO ( the borowitz report ) — ...",41.921875,41.921875,0.0,3.630889,5.133659,5.282277,3.553495,4.097359,9.357847,10.666996,NaN,NaN
4,4,yahoonews,• none sen. ben sasse on sunday said that the ...,40.188500,40.188500,0.0,5.814619,6.304806,5.182654,5.754733,4.614905,8.495536,7.279321,left-center,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,245,theborowitzreport,ATLANTA ( the borowitz report ) — president bi...,41.743000,41.743000,0.0,6.694711,3.879984,5.066907,4.406429,4.028397,12.651327,6.916868,NaN,NaN
246,246,theborowitzreport,"WELLINGTON , OHIO ( the borowitz report ) — ho...",23.266667,23.266667,0.0,5.819775,10.039607,11.211203,8.059291,6.255706,8.071219,7.168476,NaN,NaN
247,247,deneenborelli,president biden flip - flopped on the decision...,29.381667,29.381667,0.0,7.535979,5.877512,4.637794,6.587134,4.300400,7.955576,6.829192,NaN,NaN
248,248,rightwingwatch,right - wing pastor and self - proclaimed “ pr...,37.983929,37.983929,0.0,3.913609,5.162972,5.320155,4.980907,4.215590,11.476372,7.765704,left-bias,1.0


In [15]:
nela_cleaned = pd.read_csv('orientation_merge.csv')

conspiracy_pseudoscience = nela_cleaned[nela_cleaned['label_number'] == 4]
center = nela_cleaned[nela_cleaned['label_number'] == 3]
left_center = nela_cleaned[nela_cleaned['label_number'] == 2]
left_bias = nela_cleaned[nela_cleaned['label_number'] == 1]

In [16]:
conspiracy_pseudoscience_corr = conspiracy_pseudoscience.corr()
left_center_corr = left_center.corr()
center_corr = center.corr()
left_bias_corr = left_bias.corr()

display(conspiracy_pseudoscience_corr)
display(left_center_corr)
display(center_corr)
display(left_bias_corr)
display(nela_cleaned.corr())


,Unnamed: 0,Unnamed: 0.1,care_virtue,fairness_virtue,loyalty_virtue,authority_virtue,sanctity_virtue,care_vice,fairness_vice,loyalty_vice,authority_vice,sanctity_vice,label_number
Unnamed: 0,1.000000,1.000000,-0.031042,-0.031042,NaN,0.290726,0.210574,0.226787,0.220157,0.384486,-0.123450,-0.279367,NaN
Unnamed: 0.1,1.000000,1.000000,-0.031042,-0.031042,NaN,0.290726,0.210574,0.226787,0.220157,0.384486,-0.123450,-0.279367,NaN
care_virtue,-0.031042,-0.031042,1.000000,1.000000,NaN,-0.593481,-0.567600,-0.487569,-0.556828,-0.438634,0.574072,0.398969,NaN
fairness_virtue,-0.031042,-0.031042,1.000000,1.000000,NaN,-0.593481,-0.567600,-0.487569,-0.556828,-0.438634,0.574072,0.398969,NaN
loyalty_virtue,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
authority_virtue,0.290726,0.290726,-0.593481,-0.593481,NaN,1.000000,0.910978,0.945756,0.947245,0.863735,-0.864899,-0.872368,NaN
sanctity_virtue,0.210574,0.210574,-0.567600,-0.567600,NaN,0.910978,1.000000,0.923230,0.928689,0.863379,-0.847221,-0.879863,NaN
care_vice,0.226787,0.226787,-0.487569,-0.487569,NaN,0.945756,0.923230,1.000000,0.953607,0.862098,-0.871228,-0.905090,NaN
fairness_vice,0.220157,0.220157,-0.556828,-0.556828,NaN,0.947245,0.928689,0.953607,1.000000,0.848368,-0.889130,-0.865872,NaN
loyalty_vice,0.384486,0.384486,-0.438634,-0.438634,NaN,0.863735,0.863379,0.862098,0.848368,1.000000,-0.845938,-0.869311,NaN


,Unnamed: 0,Unnamed: 0.1,care_virtue,fairness_virtue,loyalty_virtue,authority_virtue,sanctity_virtue,care_vice,fairness_vice,loyalty_vice,authority_vice,sanctity_vice,label_number
Unnamed: 0,1.000000,1.000000,0.166173,0.166173,NaN,0.004636,-0.061593,-0.019929,-0.107776,-0.015785,0.045699,0.057677,NaN
Unnamed: 0.1,1.000000,1.000000,0.166173,0.166173,NaN,0.004636,-0.061593,-0.019929,-0.107776,-0.015785,0.045699,0.057677,NaN
care_virtue,0.166173,0.166173,1.000000,1.000000,NaN,-0.063623,-0.134153,0.105202,-0.166473,0.070224,0.095136,-0.069781,NaN
fairness_virtue,0.166173,0.166173,1.000000,1.000000,NaN,-0.063623,-0.134153,0.105202,-0.166473,0.070224,0.095136,-0.069781,NaN
loyalty_virtue,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
authority_virtue,0.004636,0.004636,-0.063623,-0.063623,NaN,1.000000,0.790556,0.583931,0.769487,0.740773,-0.597961,-0.698371,NaN
sanctity_virtue,-0.061593,-0.061593,-0.134153,-0.134153,NaN,0.790556,1.000000,0.733424,0.834954,0.855074,-0.689127,-0.712704,NaN
care_vice,-0.019929,-0.019929,0.105202,0.105202,NaN,0.583931,0.733424,1.000000,0.677658,0.827427,-0.729402,-0.680592,NaN
fairness_vice,-0.107776,-0.107776,-0.166473,-0.166473,NaN,0.769487,0.834954,0.677658,1.000000,0.723846,-0.651093,-0.703032,NaN
loyalty_vice,-0.015785,-0.015785,0.070224,0.070224,NaN,0.740773,0.855074,0.827427,0.723846,1.000000,-0.709040,-0.667065,NaN


,Unnamed: 0,Unnamed: 0.1,care_virtue,fairness_virtue,loyalty_virtue,authority_virtue,sanctity_virtue,care_vice,fairness_vice,loyalty_vice,authority_vice,sanctity_vice,label_number
Unnamed: 0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Unnamed: 0.1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
care_virtue,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fairness_virtue,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
loyalty_virtue,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
authority_virtue,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
sanctity_virtue,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
care_vice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fairness_vice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
loyalty_vice,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Unnamed: 0,Unnamed: 0.1,care_virtue,fairness_virtue,loyalty_virtue,authority_virtue,sanctity_virtue,care_vice,fairness_vice,loyalty_vice,authority_vice,sanctity_vice,label_number
Unnamed: 0,1.000000,1.000000,0.052763,0.052763,NaN,-0.245369,-0.248407,-0.273251,-0.240335,-0.243841,0.313698,0.334740,NaN
Unnamed: 0.1,1.000000,1.000000,0.052763,0.052763,NaN,-0.245369,-0.248407,-0.273251,-0.240335,-0.243841,0.313698,0.334740,NaN
care_virtue,0.052763,0.052763,1.000000,1.000000,NaN,-0.690096,-0.670558,-0.701212,-0.623850,-0.646047,0.578841,0.629562,NaN
fairness_virtue,0.052763,0.052763,1.000000,1.000000,NaN,-0.690096,-0.670558,-0.701212,-0.623850,-0.646047,0.578841,0.629562,NaN
loyalty_virtue,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
authority_virtue,-0.245369,-0.245369,-0.690096,-0.690096,NaN,1.000000,0.793579,0.872969,0.815931,0.877768,-0.738556,-0.825146,NaN
sanctity_virtue,-0.248407,-0.248407,-0.670558,-0.670558,NaN,0.793579,1.000000,0.901056,0.914068,0.878159,-0.750108,-0.845653,NaN
care_vice,-0.273251,-0.273251,-0.701212,-0.701212,NaN,0.872969,0.901056,1.000000,0.910691,0.899937,-0.747848,-0.859872,NaN
fairness_vice,-0.240335,-0.240335,-0.623850,-0.623850,NaN,0.815931,0.914068,0.910691,1.000000,0.854617,-0.833967,-0.862653,NaN
loyalty_vice,-0.243841,-0.243841,-0.646047,-0.646047,NaN,0.877768,0.878159,0.899937,0.854617,1.000000,-0.760492,-0.832987,NaN


,Unnamed: 0,Unnamed: 0.1,care_virtue,fairness_virtue,loyalty_virtue,authority_virtue,sanctity_virtue,care_vice,fairness_vice,loyalty_vice,authority_vice,sanctity_vice,label_number
Unnamed: 0,1.000000,1.000000,0.091081,0.091081,NaN,0.078552,0.066556,0.034531,0.018667,0.090963,0.046349,0.030896,0.123888
Unnamed: 0.1,1.000000,1.000000,0.091081,0.091081,NaN,0.078552,0.066556,0.034531,0.018667,0.090963,0.046349,0.030896,0.123888
care_virtue,0.091081,0.091081,1.000000,1.000000,NaN,-0.262168,-0.279757,-0.218656,-0.282083,-0.200600,0.425096,0.335994,-0.085052
fairness_virtue,0.091081,0.091081,1.000000,1.000000,NaN,-0.262168,-0.279757,-0.218656,-0.282083,-0.200600,0.425096,0.335994,-0.085052
loyalty_virtue,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
authority_virtue,0.078552,0.078552,-0.262168,-0.262168,NaN,1.000000,0.816180,0.814431,0.813581,0.838744,-0.714449,-0.745762,0.295181
sanctity_virtue,0.066556,0.066556,-0.279757,-0.279757,NaN,0.816180,1.000000,0.868171,0.872472,0.863500,-0.723604,-0.756687,0.227848
care_vice,0.034531,0.034531,-0.218656,-0.218656,NaN,0.814431,0.868171,1.000000,0.867379,0.851638,-0.743538,-0.773726,0.153083
fairness_vice,0.018667,0.018667,-0.282083,-0.282083,NaN,0.813581,0.872472,0.867379,1.000000,0.819375,-0.735005,-0.745390,0.105853
loyalty_vice,0.090963,0.090963,-0.200600,-0.200600,NaN,0.838744,0.863500,0.851638,0.819375,1.000000,-0.703774,-0.731981,0.185652


In [17]:
def preprocess_text(text):
    # Tokenize the text
    tokens = word_tokenize(text.lower())

    # Remove stopwords and single-character words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words and len(word) > 2]

    # Lemmatize the words
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return tokens

input_string = orientation_merge.iloc[1]['body']
word_list = input_string.split()
preprocessed_words = [preprocess_text(word) for word in word_list]

In [18]:
# Define the number of topics
num_topics = 10

# Create a dictionary mapping of word to unique id
dictionary = corpora.Dictionary(preprocessed_words)

# Create a corpus as a list of Bag of Words (BoW) representations
corpus = [dictionary.doc2bow(text) for text in preprocessed_words]

# Train an LDA model
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=15)
# Get the top 10 words for each topic
topic_words = defaultdict(list)

for topic_id, topic in lda_model.show_topics(num_topics=num_topics, num_words=10, formatted=False):
    for word, weight in topic:
        topic_words[topic_id].append((word, weight))

# Print the top 10 words for each topic
for topic_id, words in topic_words.items():
    print(f"Topic {topic_id + 1}: {', '.join([word for word, _ in words])}")
# 50 articles from each leaning

Topic 1: hundred, new, wmu, hospital, joined, choice, â, \nthousands, come, police
Topic 2: educator, quit, \nit, lapd, without, long, brook, announced, issue, resident
Topic 3: school, employee, alone, \nhundreds, \nten, \nin, million, \nmillions, make, maxwell
Topic 4: get, filed, thousand, department, team, constitution, stand, defense, blasted, end
Topic 5: state, jab, bianco, covid-19, \non, federal, inspire, mayor, together, county
Topic 6: become, mandate, \nhe, united, liberty, \nlast, across, foley, u, dog
Topic 7: \n, medical, night, ago, teacher, york, opposition, right, 30, consult
Topic 8: one, vaccine, religious, sheriff, monday, public, 100, system, kicking, front
Topic 9: exemption, maternity, majority, officer, take, elected, general, \nthe, \nnew, enforce
Topic 10: kane, week, lawsuit, health, mandatory, catanzara, opportunity, unvaccinated, quartet, asserted


In [19]:
# for word, scores in mfd_dict.items():
#     print(word)
#     print(stuff)

In [20]:
import gensim.downloader as api

# Load a pre-trained Word2Vec model (you can replace 'word2vec-google-news-300' with your chosen model)
model = api.load("word2vec-google-news-300")

In [21]:
# Initialize an empty list to store the vectors
word_vectors = {}
topic_vectors = {}
morality_dictionary = []

for topic_id, words in topic_words.items():
    topic_vectors[topic_id] = {}
    for word, _ in words:
        topic_vectors[topic_id][word] = []

# Iterate through the words and get their vectors
for word, score in mfd_dict.items():
    morality_dictionary.append(word)
    try:
        word_vectors[word] = []
        vector = model[word]  # Get the vector representation of the word
        word_vectors[word].append(vector)
    except KeyError:
        # Handle the case where the word is not in the vocabulary
        print(f"Word '{word}' not found in the vocabulary.")

# word_vectors will now contain the vector representations of the words in your list

for topic_id, words in topic_words.items():
    for word, _ in words:
        try:
            vector = model[word]  # Get the vector representation of the word
            topic_vectors[topic_id][word].append(vector)
        except KeyError:
            # Handle the case where the word is not in the vocabulary
            print(f"Word '{word}' not found in the vocabulary.")


Word 'wmu' not found in the vocabulary.
Word 'â' not found in the vocabulary.
Word '\nthousands' not found in the vocabulary.
Word '\nit' not found in the vocabulary.
Word 'lapd' not found in the vocabulary.
Word '\nhundreds' not found in the vocabulary.
Word '\nten' not found in the vocabulary.
Word '\nin' not found in the vocabulary.
Word '\nmillions' not found in the vocabulary.
Word 'covid-19' not found in the vocabulary.
Word '\non' not found in the vocabulary.
Word '\nhe' not found in the vocabulary.
Word '\nlast' not found in the vocabulary.
Word '\n' not found in the vocabulary.
Word '30' not found in the vocabulary.
Word '100' not found in the vocabulary.
Word '\nthe' not found in the vocabulary.
Word '\nnew' not found in the vocabulary.
Word 'catanzara' not found in the vocabulary.


In [22]:
from sklearn.metrics.pairwise import cosine_similarity

similar_words = {}

for topic, word_vector in topic_vectors.items():
    similar_words[topic] = []
    for word, vector in word_vector.items():
        similar_words[topic][word] = []

morality_vectors = []
for word, vector in word_vectors.items():
    morality_vectors.append(vector[0])

for topic, word_vector in topic_vectors.items():
    for article_word, article_vector in word_vector.items():
        if article_vector == []:
            pass
        else:
            sim = cosine_similarity([article_vector[0]], morality_vectors)
            similar_words[topic][article_word]

TypeError: list indices must be integers or slices, not str

In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd
import numpy as np
tfidf_vectorizer = TfidfVectorizer(analyzer="char")

similar_words = {}

for topic, word_vector in topic_vectors.items():
    similar_words[topic] = {}
    # for word, vector in word_vector.items():
    #     similar_words[topic][word] = []

for topic, topic_vector in topic_vectors.items():
    for word, vector in topic_vector.items():
        sparse_matrix = tfidf_vectorizer.fit_transform([word]+morality_dictionary)
        cosine = cosine_similarity(sparse_matrix[0,:],sparse_matrix[1:,:])
        similar_words[topic][word] = cosine
    #include cosine sim as weighting

for topic, words in similar_words.items():
    print("topic: ", topic)
    for word, similarity in words.items():
        print("word: ", word)
        top_10_indices = np.argsort(similarity)[0][-10:]
        for i in top_10_indices:
            print(morality_dictionary[i])


topic:  0
word:  hundred
denied
endorsed
shared
hatred
hide
murdered
hard
launched
punished
hand
word:  new
weapons
warning
weapon
wise
warned
wins
want
winners
winner
win
word:  wmu
lawsuits
wars
low
wise
win
war
welcomed
lawsuit
welcome
warming
word:  hospital
stop
ethical
complaints
hope
passionate
help
hostility
hopes
helps
hostile
word:  joined
injury
rejected
joy
prejudice
justified
injustice
enjoy
rejection
injured
join
word:  choice
convince
critics
nice
havoc
critic
ethical
reach
chaos
rich
chance
word:  â
bad
lies
opportunity
critic
approved
rebels
motivated
spirit
kind
violating
word:  \nthousands
hand
stronghold
dangerous
solutions
assassination
punished
tensions
punishments
outstanding
honest
word:  come
combat
welcomed
welcome
crime
recommended
committed
commitments
commitment
commit
recommend
word:  police
conflict
popular
peace
violence
prosecution
complaints
prospect
complaint
special
collapse
topic:  1
word:  educator
credit
torture
encourage
protected
accused
creat

In [ ]:
# import fasttext
# import fasttext.util
# import numpy as np

# # download English pretrained model
# fasttext.util.download_model('en', if_exists='ignore')
# ft = fasttext.load_model('cc.en.300.bin')

# def cos_sim(a, b):
#     """Takes 2 vectors a, b and returns the cosine similarity according 
#     to the definition of the dot product
#     (https://masongallo.github.io/machine/learning,/python/2016/07/29/cosine-similarity.html)
#     """
#     dot_product = np.dot(a, b)
#     norm_a = np.linalg.norm(a)
#     norm_b = np.linalg.norm(b)
#     return dot_product / (norm_a * norm_b)

# def compare_word(w, words_vectors):
#     """
#     Compares new word with those in the words vectors dictionary
#     """
#     vec=ft.get_sentence_vector(w)
#     return {w1:cos_sim(vec,vec1) for w1,vec1 in words_vectors.items()}

# # define your word list
# for topic_id, words in topic_words.items():
#     # word_list = words
#     # compute words vectors and save them into a dictionary.
#     # since there are multiwords expressions, we use get_sentence_vector method
#     # instead, you can use  get_word_vector method
#     words_vectors={w:ft.get_sentence_vector(w) for  w in words}

AttributeError: module 'fasttext' has no attribute 'util'